# Fine tune a la red de PlantVillage 
Usamos la MobileNetV2 para validar el concepto de fine tune con este pre entrenamiento

In [1]:
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import scipy 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import IPython.display as display
import time

keras = tf.keras
print(tf.__version__)



2.0.0


In [3]:
data_dir = './dataset/trigo/'
data_dir = pathlib.Path(data_dir)

In [6]:
N = 0  # total files
for dirpath, dirnames, filenames in os.walk(data_dir):    
    dirpath = dirpath.split("/")[-1]
    if dirpath != ".ipynb_checkpoints":
        N_c = len(filenames)
        N += N_c
        print( dirpath+ ": -> " + str(N_c))
print( "Total Files " + str(N) )

trigo: -> 0
mancha_foliar: -> 122
oidio: -> 102
sano: -> 113
roya_amarilla: -> 70
roya_anaranjada: -> 89
Total Files 496


In [5]:
image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png')))
image_count

496

In [25]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  validation_split=0.15,                            
                                                                  shear_range=0.2,
                                                                  zoom_range=0.2,
                                                                  horizontal_flip=True,
                                                                  rotation_range=20
                                                                 )

In [26]:
BATCH_SIZE = 16
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil((image_count*0.9)/BATCH_SIZE)
int(STEPS_PER_EPOCH)

28

In [27]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != ".ipynb_checkpoints"])
CLASS_NAMES

array(['mancha_foliar', 'oidio', 'sano', 'roya_amarilla',
       'roya_anaranjada'], dtype='<U15')

In [28]:
train_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "training",
                                                     classes = list(CLASS_NAMES))

Found 424 images belonging to 5 classes.


In [29]:
valid_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "validation",
                                                     classes = list(CLASS_NAMES))

Found 72 images belonging to 5 classes.


## MobileNetV2

In [23]:
model_p = tf.keras.models.load_model('models/plant_village_MobileNetV2_trained_01.h5')

In [24]:
model_p.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 27)                34587     
Total params: 2,292,571
Trainable params: 2,258,459
Non-trainable params: 34,112
_________________________________________________________________


In [25]:
# x2 remove 2 last layers
model_p.pop()
model_p.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
Total params: 2,292,571
Trainable params: 2,258,459
Non-trainable params: 34,112
_________________________________________________________________


In [26]:
#global_average_layer = tf.keras.layers.GlobalAveragePooling2D(model_p.layers[-2].output)
prediction_layer = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")

In [27]:
model_p.add(prediction_layer)

In [28]:
len(model_p.layers)

3

In [29]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in model_p.layers[:1]:
    layer.trainable =  False

In [30]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001)
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [31]:
base_learning_rate = 0.002

model_p.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=base_learning_rate, beta_1=0.9, beta_2=0.999),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
total_epochs = 30
start_time = time.time()
history = model_p.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [ reduce_lr]
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/30
14/14 [==============================] - 29s 2s/step - loss: 1.5174 - accuracy: 0.3371 - val_loss: 1.4108 - val_accuracy: 0.4583
Epoch 2/30
14/14 [==============================] - 27s 2s/step - loss: 1.3292 - accuracy: 0.4598 - val_loss: 1.7104 - val_accuracy: 0.3958
Epoch 3/30
14/14 [==============================] - 27s 2s/step - loss: 1.3066 - accuracy: 0.4844 - val_loss: 1.6991 - val_accuracy: 0.3958
Epoch 4/30
14/14 [==============================] - 26s 2s/step - loss: 1.2275 - accuracy: 0.5022 - val_loss: 1.6507 - val_accuracy: 0.3542
Epoch 5/30
14/14 [==============================] - 27s 2s/step - loss: 1.1763 - accuracy: 0.5045 - val_loss: 1.7379 - val_accuracy: 0.3750
Epoch 6/30
14/14 [==============================] - 29s 2s/step - loss: 1.1296 - accuracy: 0.5513 - val_loss: 1.6053 - val_accuracy: 0.3750
Epoch 7/30
14/14 [==============================] - 27s 2s/step - loss: 1.1436 - accuracy: 0.5580 - val_loss: 1.5676 - val_accuracy: 0.3542
Epoch 8/30
14/14 [==

KeyboardInterrupt: 

## Xception

In [59]:
model_x = tf.keras.models.load_model('models/plant_village_Xception_trained_01.h5')
prediction_layer = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_x.pop()
model_x.add(prediction_layer)

Freeze xception model 70 layers 

In [60]:
for layer in model_x.layers[0].layers: 
    layer.trainable = False

In [61]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001)
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3 
                                         #, baseline=0.5 , restore_best_weights=True
                                        )
filepathM = "models/plant_village_trigo_Xception.{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = filepathM, 
                                                          save_best_only=True , monitor='val_accuracy', mode='max')

In [62]:
base_learning_rate = 0.0003
model_x.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [63]:
total_epochs = 30
start_time = time.time()
history_x = model_x.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [reduce_lr , checkpoint]
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/30
27/27 [==============================] - 30s 1s/step - loss: 1.5732 - accuracy: 0.3231 - val_loss: 1.5535 - val_accuracy: 0.3056
Epoch 2/30
27/27 [==============================] - 25s 927ms/step - loss: 1.3693 - accuracy: 0.4009 - val_loss: 1.5920 - val_accuracy: 0.2222
Epoch 3/30
27/27 [==============================] - 25s 942ms/step - loss: 1.2991 - accuracy: 0.4599 - val_loss: 1.5614 - val_accuracy: 0.3056
Epoch 4/30
27/27 [==============================] - 25s 929ms/step - loss: 1.2206 - accuracy: 0.5094 - val_loss: 1.6807 - val_accuracy: 0.2222
Epoch 5/30
27/27 [==============================] - 25s 934ms/step - loss: 1.2412 - accuracy: 0.4858 - val_loss: 1.6219 - val_accuracy: 0.2222
Epoch 6/30
27/27 [==============================] - 25s 926ms/step - loss: 1.1911 - accuracy: 0.5566 - val_loss: 1.6684 - val_accuracy: 0.2500
Epoch 7/30
27/27 [==============================] - 25s 931ms/step - loss: 1.1966 - accuracy: 0.5330 - val_loss: 1.5779 - val_accuracy: 0.2639
Ep

KeyboardInterrupt: 

## VGG19

In [64]:
model_v = tf.keras.models.load_model('models/plant_village_VGG19_trained_01.h5')
prediction_layer_v = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_v.pop()
model_v.add(prediction_layer_v)

In [65]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001)
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3 
                                         #, baseline=0.5 , restore_best_weights=True
                                        )
filepathM = "models/plant_village_trigo_vgg.{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = filepathM, 
                                                          save_best_only=True , monitor='val_accuracy', mode='max')

In [66]:
base_learning_rate = 0.0003
model_v.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [67]:
total_epochs = 30
start_time = time.time()
history_v = model_v.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [reduce_lr , checkpoint]
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/30
27/27 [==============================] - 28s 1s/step - loss: 2.5442 - accuracy: 0.1981 - val_loss: 1.6067 - val_accuracy: 0.1806
Epoch 2/30
27/27 [==============================] - 24s 884ms/step - loss: 1.6114 - accuracy: 0.2382 - val_loss: 1.5954 - val_accuracy: 0.2083
Epoch 3/30
27/27 [==============================] - 24s 876ms/step - loss: 1.6457 - accuracy: 0.2335 - val_loss: 1.5920 - val_accuracy: 0.2639
Epoch 4/30
27/27 [==============================] - 24s 875ms/step - loss: 1.6065 - accuracy: 0.2453 - val_loss: 1.5901 - val_accuracy: 0.2778
Epoch 5/30
27/27 [==============================] - 23s 850ms/step - loss: 1.7589 - accuracy: 0.2524 - val_loss: 1.6070 - val_accuracy: 0.2500
Epoch 6/30
27/27 [==============================] - 23s 862ms/step - loss: 1.6444 - accuracy: 0.2642 - val_loss: 1.6084 - val_accuracy: 0.2500
Epoch 7/30
27/27 [==============================] - 23s 867ms/step - loss: 1.5785 - accuracy: 0.2618 - val_loss: 1.5580 - val_accuracy: 0.3611
Ep

In [70]:
history_v2 = model_v.fit_generator(train_data_gen,
                         epochs=50,
                         initial_epoch = 30,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [reduce_lr , checkpoint]
                        )

Epoch 31/50
27/27 [==============================] - 28s 1s/step - loss: 0.7953 - accuracy: 0.7170 - val_loss: 1.8531 - val_accuracy: 0.5000
Epoch 32/50
27/27 [==============================] - 23s 862ms/step - loss: 0.7123 - accuracy: 0.7264 - val_loss: 1.6747 - val_accuracy: 0.4583
Epoch 33/50
27/27 [==============================] - 23s 867ms/step - loss: 0.6690 - accuracy: 0.7500 - val_loss: 1.9621 - val_accuracy: 0.4722
Epoch 34/50
27/27 [==============================] - 24s 881ms/step - loss: 0.7391 - accuracy: 0.7241 - val_loss: 1.6452 - val_accuracy: 0.5694
Epoch 35/50
27/27 [==============================] - 23s 857ms/step - loss: 0.6925 - accuracy: 0.7571 - val_loss: 1.7121 - val_accuracy: 0.5556
Epoch 36/50
27/27 [==============================] - 23s 866ms/step - loss: 0.6445 - accuracy: 0.7783 - val_loss: 1.7197 - val_accuracy: 0.4861
Epoch 37/50
27/27 [==============================] - 23s 864ms/step - loss: 0.6447 - accuracy: 0.7783 - val_loss: 2.1004 - val_accuracy: 0.

In [71]:
model_v = tf.keras.models.load_model('models/plant_village_trigo_vgg.39-0.60.h5')

In [72]:
for layer in model_v.layers[0].layers: 
    layer.trainable = False

In [73]:
base_learning_rate = 0.001
model_v.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [75]:
history_v2 = model_v.fit_generator(train_data_gen,
                         epochs=70,
                         initial_epoch = 50,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [reduce_lr , checkpoint]
                        )

Epoch 51/70
27/27 [==============================] - 27s 1s/step - loss: 0.3889 - accuracy: 0.8585 - val_loss: 1.3692 - val_accuracy: 0.5417
Epoch 52/70
27/27 [==============================] - 23s 856ms/step - loss: 0.3671 - accuracy: 0.8585 - val_loss: 1.4253 - val_accuracy: 0.5417
Epoch 53/70
27/27 [==============================] - 24s 873ms/step - loss: 0.4027 - accuracy: 0.8349 - val_loss: 1.5465 - val_accuracy: 0.6111
Epoch 54/70
27/27 [==============================] - 23s 868ms/step - loss: 0.3543 - accuracy: 0.8844 - val_loss: 1.5508 - val_accuracy: 0.6389
Epoch 55/70
27/27 [==============================] - 23s 853ms/step - loss: 0.3805 - accuracy: 0.8726 - val_loss: 1.5130 - val_accuracy: 0.5417
Epoch 56/70
27/27 [==============================] - 23s 869ms/step - loss: 0.3570 - accuracy: 0.8844 - val_loss: 1.4954 - val_accuracy: 0.5972
Epoch 57/70
27/27 [==============================] - 23s 845ms/step - loss: 0.3464 - accuracy: 0.8774 - val_loss: 1.5567 - val_accuracy: 0.

In [79]:
model_v = tf.keras.models.load_model('models/plant_village_trigo_vgg.59-0.65.h5')

In [80]:
for layer in model_v.layers[0].layers: 
    layer.trainable = False

In [81]:
base_learning_rate = 0.0001
model_v.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_vfine = model_v.fit_generator(train_data_gen,
                         epochs=90,
                         initial_epoch = 70,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [reduce_lr , checkpoint]
                        )

Epoch 71/90
27/27 [==============================] - 27s 1s/step - loss: 0.3666 - accuracy: 0.8774 - val_loss: 1.3845 - val_accuracy: 0.5972
Epoch 72/90
27/27 [==============================] - 23s 867ms/step - loss: 0.3596 - accuracy: 0.8679 - val_loss: 1.5458 - val_accuracy: 0.5833
Epoch 73/90
27/27 [==============================] - 23s 864ms/step - loss: 0.3629 - accuracy: 0.8632 - val_loss: 1.6602 - val_accuracy: 0.6111
Epoch 74/90
27/27 [==============================] - 23s 851ms/step - loss: 0.3549 - accuracy: 0.8656 - val_loss: 1.5609 - val_accuracy: 0.6111
Epoch 75/90
27/27 [==============================] - 23s 851ms/step - loss: 0.3353 - accuracy: 0.8750 - val_loss: 1.6581 - val_accuracy: 0.5972
Epoch 76/90
27/27 [==============================] - 24s 874ms/step - loss: 0.4087 - accuracy: 0.8491 - val_loss: 1.3717 - val_accuracy: 0.6389
Epoch 77/90
27/27 [==============================] - 23s 859ms/step - loss: 0.3316 - accuracy: 0.8844 - val_loss: 1.4074 - val_accuracy: 0.

## Try lr finder

In [3]:
from keras_lr_finder import LRFinder

Using TensorFlow backend.


In [6]:
CLASS_NAMES = 5

In [7]:
model_x = tf.keras.models.load_model('models/plant_village_Xception_trained_01.h5')
prediction_layer = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_x.pop()
model_x.add(prediction_layer)

ResourceExhaustedError: OOM when allocating tensor with shape[1,1,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: block3_sepconv2_1/pointwise_kernel/Initializer/random_uniform/